If you follow the instruction at [Setup.ipynb](https://github.com/physhik/baseball_lab/blob/master/practice/Setup.ipynb), you are already at /baseball_lab/src/ directory. Now import the functions at the directory and excute it.


For who forgot the setup, let me repeat the steps. However, I won't git clone the baseball_lab again since it is already at your /gdrive/'My drive' directory if you followed the setup.ipynb once.

In [ ]:
from __future__ import division
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
!ls

In [4]:
cd gdrive/'My Drive'/baseball_lab/src

/content/gdrive/My Drive/baseball_lab/src


In [3]:
from metric import *

Now we are ready to proceed the next step. 

The main work of this repo is to make new derivatives from FIP's idea. The data frame which contains the new stats can be obtained since 2015, when the statcast data are open to public. If you want to download csv file (can be open by EXCEL spreadsheet), do the following. 

In [4]:
df = new_metric(2018, 2018, '2019-03-01', '2019-10-03')

size of con, brl, con_wohr: 827 764 827
fagraphs id and name removed: 15541.0 Tyler Rogers with innings 17.2
fagraphs id and name removed: 17995.0 Logan Webb with innings 39.2
fagraphs id and name removed: 20387.0 Anthony Kay with innings 14.0
fagraphs id and name removed: 19269.0 T.J. Zeuch with innings 22.2
fagraphs id and name removed: 19959.0 Jesus Luzardo with innings 12.0
fagraphs id and name removed: 19343.0 A.J. Puk with innings 11.1
fagraphs id and name removed: 18251.0 Art Warren with innings 5.1
fagraphs id and name removed: 15507.0 Hunter Harvey with innings 6.1
fagraphs id and name removed: 20367.0 Brusdar Graterol with innings 9.2
fagraphs id and name removed: 12874.0 Danny Hultzen with innings 3.1
fagraphs id and name removed: 15256.0 Cole Sulser with innings 7.1
fagraphs id and name removed: 20151.0 James Karinchak with innings 5.1
fagraphs id and name removed: 15538.0 Eric Yardley with innings 11.2
fagraphs id and name removed: 19268.0 Justin Dunn with innings 6.2
fagr

df is the dataframe of the fangraphs stats, and part of statcast stats and the new metric suggested by my perspective. Let me introduce the main idea of it. 






### Fielding independent pitching stat



FIP (fielding independent pitching) could be one of most famous pitching stats in recent 10 years. So intuitive and not too hard to calculate.

FIP formula has a constant term about 3.2 (varying up to the league ERA).

If I remember correctly, the constant comes from the bip and was suggested to be a fixed number because our understanding to batted ball was still unclear.

Statcast now provides many good informations of the batted balls. Thus, we would want to really fielding independent pitching stats finally by using xwOBAcon or some other statcast stats else. I used xwOBAcon and rescaled the constant term to league ERA. You can see the formula at a python file in the Github repo. I temporarily named it nFIP in my project files.

HR is also dependent on the environments such as the stadium, weathers, so we want to use barrels instead of HR as the represtative of the well-crushed hit. I named it xnFIP in my project files.

One of the motives of this project is that FIP underestimates ground ball pitchers and the constant term about 3.2 is too big. Of course, strikeouts are really good. However, how do we estimate the value of weakly contacted ball in WAR?

The 2nd motive is ... if you check the variance of ERA of the players in the league compared to that of FIP. FIP's variance is much smaller. And the variances of xFIP, SIERA are much much smaller. That's why people thought FIP, xFIP, SIERA have some predictive potential. However, those stats just predict less boldly, and that's why they are wrong less to anticipate.

If the fixed number 3.2 varies, the variance of nFIP become larger and nFIP still yields better correlations to next year pitching stats. I think this caused by good property of the new stats to put the value in weak contacts. There are so much room to be adjusted in the stats, but hope this is a right direction to evaluate pitchers' ability.

I also want to discuss if rescaling FIP to ERA makes sense. Why not RA? I want to call it rFIP here, and made derivatives too. I prefer R because ER is influenced by error more than R. I wanted to avoid the situatioin HR, SO, BB are included only to FIP, not to ERA because of errors.

About xFIP, I think giving the same HR rate to the fly balls is rather old fashioned since the fly ball exit velocities have been measured.

---



In [6]:
df.head(20)

,Name,Team,Age,W,L,ERA,G,WAR,GS,CG,...,xnFIPwo,nFIPwo,rnFIP,rnFIPwo,rxnFIP,rxnFIPwo,cnfip,cnfipwo,crnfip,crnfipwo
mlb_id,,,,,,,,,,,,,,,,,,,,,
543037,Gerrit Cole,Astros,28.0,20.0,5.0,2.500785,33.0,7.4,33.0,0.0,...,1.719829,2.334556,2.750421,2.659960,2.135694,2.045233,2.990485,2.909125,3.324990,3.234529
594798,Jacob deGrom,Mets,31.0,11.0,8.0,2.426471,32.0,7.0,32.0,0.0,...,2.341080,2.390133,2.521812,2.717799,2.472759,2.668746,2.753078,2.929349,3.061028,3.257015
458681,Lance Lynn,Rangers,32.0,16.0,11.0,3.672000,33.0,6.8,33.0,0.0,...,3.456625,3.265436,3.398539,3.640360,3.589727,3.831549,3.134342,3.351836,3.484939,3.726760
453286,Max Scherzer,Nationals,34.0,11.0,7.0,2.924565,27.0,6.5,27.0,0.0,...,2.755599,2.628214,2.713060,3.007874,2.840445,3.135258,3.129017,3.394172,3.479018,3.773831
434378,Justin Verlander,Astros,36.0,21.0,6.0,2.582960,34.0,6.4,34.0,2.0,...,2.190131,2.630896,3.155482,2.919160,2.714717,2.478394,2.789633,2.577085,3.101671,2.865348
450203,Charlie Morton,Rays,35.0,16.0,6.0,3.051370,33.0,6.1,33.0,0.0,...,2.950446,2.783425,2.785590,3.139588,2.952611,3.306609,2.865725,3.184110,3.186275,3.540273
544931,Stephen Strasburg,Nationals,30.0,18.0,6.0,3.315789,33.0,5.7,33.0,0.0,...,2.848907,2.890288,3.096363,3.209304,3.054981,3.167922,2.750432,2.852011,3.058085,3.171027
669456,Shane Bieber,Indians,24.0,15.0,8.0,3.275272,34.0,5.6,33.0,3.0,...,3.813696,3.570958,3.910027,3.958389,4.152765,4.201127,3.420151,3.463648,3.802717,3.851079
425844,Zack Greinke,- - -,35.0,18.0,5.0,2.932907,33.0,5.4,33.0,0.0,...,3.326590,2.953986,3.009895,3.283872,3.382498,3.656476,2.702780,2.949194,3.005103,3.279080


*It* uses the master.csv file to find mapping between fangraphs player id and mlb.com player id. Some mlb ids are missing so had to remove some fangraph id and player stats. If the innings are not ignorable, you might need to find a mapping by hand. 

In [8]:
df.to_csv('../downloads/2019.csv')

If you excuted the above code line, you will find the 2018.csv file in your download directory. 